# OpenStreetMap Data of Shakopee, MN



### Information about the Data:

This project wrangles the data of Shakopee, MN area. I have lived in Minnesota for nearly a decade. So It's like my hometown. As the "Shakopee.osm" file is 70 MB only, I have used the complete file for analysis. It's downloaded from openstreetmap.org. It is an open-source project, which provides a free map for the entire world from volunteer-entered data. OpenStreetMap data is freely available to download in many formats and it gives an ideal opportunity to practice the art of data wrangling.
#### Why?
This Analysis is a requirement for Data Wrangling Project from Udacity's Data Analyst Nano Degree.
####  Purpose:
The file is very big and it contains thousands of rows. So editing and updating the wrong data is a difficult task. And also it's  difficult to work manually on the data provided. 
So we have to automate the process of:
* Downloading and Extraction of OSM, XML files
* Exploring, auditing and cleaning the data.
* After cleaning extracting the data into CSV files
* Connecting to database
* Running SQL queries
* And report your findings

##### Problems : State:

* From the auditing we got to know that problematic state values are 'Mn', 'Minnesota', 'mn', 'MN'.
* Correct representation of the state should be with two capital letters.
*  Ex: MN
* So I have created a function to update the problematic "state abbreviations"  to "correct state abbreviations"
#### The Update functions updates the states as follows:
```
Minnesota => MN
mn => MN
Mn => MN
```

##### Problems : City:

* Problematic cities are {'burnsville', 'Chaska MN', 'shakopee'}.
* As the names of the cities are having the inconsistent abbreviation, I have created an "update" function to allow the following changes.
* The "update" function removed "MN" from the city name.
* And all first letters of the city are capitalized
#### The Update functions updates the cities as follows:
```
('Chaska MN', '=>', 'Chaska')
('shakopee', '=>', 'Shakopee')
('burnsville', '=>', 'Burnsville')
```

##### Problems : Phone numbers:

* From the exploration and audition we understood that, phone numbers are inconsistent. They should be in the format of xxx-xxx-xxxx
* Ex: 952-200-4565
* Alphanumeric phone numbers are also present in the data.
* Ex: 800-rent-a- car
#### Update function gives the results as following:
```
(952) 496-8200 => 952-496-8200
+1 952 8923456 => 952-892-3456
952-445-7733 => 952-445-7733
+1 952 2337319 => 952-233-7319
+1 (952) 440-8845 => 952-440-8845
800-rent-a-car => 800--
```
#### Observations:
The test above confirms that the phone numbers are cleaned except the alphanumeric phone number.
Ex: 1-800-rent-a-car
#### Solution:
This can be cleaned by creating a new python function or program.
* Different regex pattern can be given for compiling.
* We can assign numbers to the letters from A-Z or a-z. 

#####  Problems : Street Names : 

* After exploring the Shakopee.osm file, I understood that we have to fix the abbreviated street names consistently. 
* This is one of the major issues I have worked on. As they are inconsistent:
* I found that there were different formats (for example, Ave, AVE, South, S, Dr, drive).
* To solve this, I have capitalized the first letter of every word in the street.
* There were a few street names in the format of numbers, number and word combinations like 5, HWY 13, Minnesota 13 S #101.
* So we have to clean them and make them consistent:
* The Update function below addresses many issues like
* Changes S to South, E to East
* Blvd to  Boulevard
* Ave to Avenue
* SE to South East etc..
#### The update function for Streets:
```
    for st_type, ways in st_types.items():
    for name in ways:
        for each in name.split():
            if each in mapping_streets.keys():
                better_name = update_name(name, mapping_streets)
                print(name, "=>", better_name)
```                
#### Results:
```
Anna Trail SE => Anna Trail SOuth East
Hwy 13 SE => Highway 13 SOuth East
1st Ave East => 1st Avenue East
Minnesota 13 S #101 => Minnesota 13 South #101
HWY 13 => Highway 13
Highway 13 S => Highway 13 South
Raspberry Ridge Road NW => Raspberry Ridge Road North West
```
#### Observations:
* Not sure about the follwing pattern, Whether it's correct or not? 
* Minnesota 13 South #101 (Ends with a number)
* Highway 13 (correct)
* North Chestnut St;S Chestnut Street is not correct. AS it's not under addr:street tag & it's under
```tag k="name" v="N Chestnut St; S Chestnut Street"
```
* Remaining street tags in osm file are tagged like below:
```
tag k="addr:street" v="17th Ave"
```
I have tried to update my function with "tag k="name" in is_streetname function:
* It's converting us bank to united states high way bank.
* Which is not correct.
* So I am not including that  k value in my analysis.

The following format is correct. So I think for these parts cleaning is not necessary)
* State Highway 13 => State Highway 13
* County Road 42 => County Road 42
#### Solution:
A few street names are included in "tag k = "name", we can further write a different program to change it to only streets by taking more parameters into account. If it still shows a problem in updating, we can delete those records or information from our data and can proceed with further analysis.

Ex:
* As I have the problem in changing the following street name, I want to see the tag where it belongs to:
* ('North Chestnut St;S Chestnut Street', 1),
* We got the information from  querying as following:
* 1136508445, 'street', 'North Chestnut St;S Chestnut Street', 'addr'),
```
# Location of wrongly abbreviated streets and other details:
wrong_abb_st_info_more  = cursor.execute("""SELECT * 
             FROM (SELECT * FROM nodes_tags
             UNION ALL 
             SELECT * FROM ways_tags) tags
             WHERE tags.id = 1136508445""").fetchall()
                                
```
```
[(1136508445, 'city', 'Chaska', 'addr'),
 (1136508445, 'housenumber', '503', 'addr'),
 (1136508445, 'postcode', '55318', 'addr'),
 (1136508445, 'street', 'North Chestnut St;S Chestnut Street', 'addr'),
 (1136508445, 'amenity', 'fast_food', 'regular'),
 (1136508445, 'name', "Domino's", 'regular')]
```
* By looking at the above information we understood that it's a fast food restaurant. 
* I googled the address and I got the information as:
* 503 N Chestnut St, Chaska, MN 55318-1925
* So it should be N Chestnut St.
*  Using a new python function or program we can deal with this

##### Problems : Postal codes:

* Incorrect zip codes or Invalid zip codes in data are as following.
* {'55379; 55378', '55044-5546', '822', '55315;55318', '55337-4236', '55337-2912'}
* Shakopee area zip code covers only 1 zip code i.e 55379.
* The other zip codes belongs to the nearby cities of Shakopee, MN.
#### Update function gives the results as following```
for element in get_element(OSMFILE):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if is_postcode(tag):
                if tag.attrib['v'] not in expected_postcodes:
                    print(tag.attrib['v'], "=>", update_postcode(tag.attrib['v']))

    ```
##### Results:

```
55044-5546 => 55044
822 => Invalid
55315;55318 => 55315
55379; 55378 => 55379
55379; 55378 => 55379
55337-4236 => 55337
55337-2912 => 55337

```
#### Observations:
A zip code is a 5 digit number, So 822 is invalid.

#### Solution:
We can explore more about this in the SQL querying section.

#### Miscellaneous
* We can also explore, audit and clean county names, cuisine type, restaurants and cafe for better analysis.
*  I have also created a pie chart in "SQL querying section" file for users, contributions, and related percentages.
* We can also run more queries like restaurants and their locations using latitudes and longitudes. 
* And plot them on a scatter plot to show in which city or place restaurants are more. 
* Plotting would have been easier if the cuisine values have been cleaned. 
* And this can also be applied to worship places also.

## Connecting to database and querying Data

##### Number of nodes:

```
cursor.execute("SELECT COUNT(*) FROM nodes")
total_nodes_tags = cursor.fetchall()
total_nodes_tags
```

```
[(325769,)]```

```
cursor.execute("SELECT COUNT(*) FROM ways")
total_nodes_tags = cursor.fetchall()
total_nodes_tags
```

```
[(32110,)]
```

```
cursor.execute("SELECT COUNT(*) FROM ways_nodes")
total_nodes_tags = cursor.fetchall()
total_nodes_tags
```

```
[(369849,)]
```

```
#These values are same as the one obtained from the exploration of dataset "Top Level Tags" 

defaultdict(<class 'int'>, {'note': 1, 'meta': 1, 'bounds': 1, 'node': 325769, 'tag': 162406, 'nd': 369849, 'way': 32110, 'member': 4967, 'relation': 152, 'osm': 1})
procesing_time:  3.534 s
```

## Number of unique users

```
cursor.execute("""SELECT COUNT(DISTINCT(e.uid))
             FROM (SELECT uid from nodes UNION ALL SELECT uid FROM ways) e""")
users = cursor.fetchall()
users
```


```
[(373,)]
We got the same value for unique users (373) as above in exploration and audition phase.

```

```
top10_contributors = cursor.execute("""SELECT e.user, COUNT(*) as num 
             FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e 
             GROUP BY e.user 
             ORDER BY num DESC 
             LIMIT 10""").fetchall()

```

![title](img/Users and no of posts.png)

Omnific tops the contribution and his total contribution are 71,874.
And the percentage of contributions in the figure below.

![title](img/users and contribution.png)

```
#Total count of individual cities cities near to Shakopee, MN
cities = cursor.execute("""SELECT tags.value, COUNT(*) as count
                        FROM (SELECT * FROM nodes_tags
                              UNION ALL 
                              SELECT * FROM ways_tags) tags 
                        WHERE tags.key = 'city' 
                        GROUP BY tags.value 
                        ORDER By count DESC""").fetchall()
```

#### Got the results as following:

```
[('Burnsville', 5089),
 ('Lakeville', 1272),
 ('Chaska', 94),
 ('Shakopee', 54),
 ('Prior Lake', 32),
 ('Bloomington', 26),
 ('Eden Prairie', 11),
 ('Savage', 9),
 ('Chanhassen', 2),
 ('Victoria', 1)]
```

Burnsville city near Shakopee, MN is popular.

#### Zipcodes:

```
#Count of post codes near Shakopee, MN area.
#Which gives the information of zip code and popularity.
postcodes = cursor.execute("""SELECT tags.value, COUNT(*) as count
                          FROM (SELECT * FROM nodes_tags
                                UNION ALL SELECT * FROM ways_tags) tags 
                          WHERE tags.key = 'postcode' or 'postal_code'
                          GROUP BY tags.value 
                          ORDER By count DESC""").fetchall()
```

#### Results:

```
[('55337', 3022),
 ('55306', 2083),
 ('55044', 1270),
 ('55318', 93),
 ('55379', 44),
 ('55372', 30),
 ('55438', 14),
 ('55378', 13),
 ('55347', 7),
 ('55344', 5),
 ('55420', 4),
 ('55431', 3),
 ('55317', 2),
 ('55437', 2),
 ('55386', 1),
 ('Invalid', 1)]
```

55337, 55306 zip codes belong to Burnsville, MN area. So It is the most popular city.

* We can see that there is an invalid code.
* After querying I got the information as follows:
* [(2390325110, 'postcode', 'Invalid', 'addr')]

```
# Give the id directly and see if you get any info
missing_postcodes_info_more = cursor.execute("""SELECT * 
             FROM (SELECT * FROM nodes_tags
             UNION ALL 
             SELECT * FROM ways_tags) tags
             WHERE tags.id = 2390325110""").fetchall()
```


##### Results:

```
[(2390325110, 'postcode', 'Invalid', 'addr'),
 (2390325110, 'street', 'West 98th Street', 'addr'),
 (2390325110, 'name', 'Discland', 'regular'),
 (2390325110, 'shop', 'music', 'regular'),
 (2390325110, 'website', 'http://disclandonline.com/', 'regular')]
```

* I googled the website "http://disclandonline.com/ ", street 'West 98th Street' and got the zip code as 55420. 
* 822 W 98th St Bloomington, MN 55420 is the address. 
* I am not cleaning it now. 
* But we can write a program to replace the invalid zip code with 55420 in exploration and audition phase.

### Suggestions

* If you carefully observe the above results data is not properly abbreviated or entered.
* Data cleaning can be influenced greatly by a good Data entry interface design
* I.e At stages from data collection and entry, through transformation.
* So my suggestion is to have a good data entering interface management system. 
* Most of the problems related to bad data will be solved by this. Although it's not a one time process. It'an iterative process.

* For example: 
* Open the website Openstreetmap.org
* These were the screenshots of entering the data into the openstreetmap.org.  * If you have any problems, Check the "img" folder for the screenshots.
* I have created a username and password and tried to enter the information like other contributors. I have tried to enter the values in the default fields for the following.

![title](img/problems in interface1.png)

![title](img/problems in interface 2.png)

![title](img/problems in interface 3.png)

###### The actual behavior of the interface:
* Looks like the software is accepting any letter, number, character for many of the fields.
* Playground name field starts with small letters
* Name field starts with small letters.
* Postcode field allows letters, numbers, and combination too.
* Fax field allows letters and numbers
* Unit field allows special characters
* City name field allows a State Name.
* Age field allows any character or number.

###### Expected behavior of the interface:
*When we develop a data entering interface:
* When we enter values in the fields, They should have some restrictions like:
* Name should start with uppercase letter
* Age should be in numbers
* Postcode should be in numbers
* The phone number should be in numbers
* Fax should be in numbers.
* City field should not accept a State name.
* Auto-population of cities, states, countries will help.
* Streets can include numbers and letters both.
* After entering the wrong data for at least important fields like "age", it should disable the upload field option.
* But It allows the user to update the changes.
* For a better data: Review from an expert for every contribution and approval helps.

Benefits:
* Data entered will be consistent and easily understandable.
* It helps in cleaning the data easily.
* It saves more time for the users, who contribute to the maps and also who analyzes the data.
* Clean data and accurate data means better predictions.

Anticipated Problems:
* For each and every country the fields and the values for the data will be different. So keeping everything in a single format is not a good idea.
* Ex: 
* Phone numbers in the US have a 10-digit format.
* Phone numbers in India for cell numbers it's a 10-digit format.
* Phone numbers in India for landline numbers it's 11-digit format.
* As people in different countries speak the different languages, People will try to edit maps in their own language. 
* So if we take large datasets of 2 different countries, if the languages are different, it's really difficult to analyze the data.
* So again maintaining a consistent format for all over the world is difficult. 
* But we can be country specific.

## Conclusion

I have tried to clean few variables like cities, streets, states, postal codes and phone numbers. Remaining variables like Counties, Cuisine, Hospitals etc.. also needs exploration, audition, and cleaning. So we can say that the analysis of the data of Shakopee, MN is still incomplete. 

For example, if we take " k value" as postcode only. It's ignoring many values tagged or named differently like postal_code, tiger_zip_code1, tiger_zip_right1 and tiger_zip_left1 etc...
These values can also be included for further investigation and cleaning.
So that important places related to these zip codes can be included in our analysis.

If the zip codes were tagged under one name, it would be easier for our analysis.

So as in the above suggestion, a better Data Entry interface helps us keep the data in a format which is easily understandable and analyzed by all.

###  Reference:


* Udacity : Data Analyst Nano Degree Lessons And quizzes

* Stackoverflow for solving problems

* XML Extraction code( get_XML_data )and Doubts:Udacity Connect Session Lead: Dylan Lennard
* https://stackoverflow.com/questions/4299675/python-script-to-convert-from-utf-8-to-ascii
* https://stackoverflow.com/questions/38337918/plot-pie-chart-and-a-table-using-matplotlib-in-pandas-dataframe

* https://stackoverflow.com/questions/42678327/python-matplotlib-add-count-number-on-top-of-the-bar
* https://matplotlib.org/devdocs/tutorials/introductory/pyplot.html#sphx-glr-tutorials-introductory-pyplot-py